In [1]:
import QDMPy.data_loading as data_loading
import QDMPy.fit_interface as fit_interface
import QDMPy.fit_plots as fit_plots

import matplotlib
import matplotlib.pyplot as plt

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

# Load the data in

In [3]:
options_dict = {
    # Argus: C:\\src\\QDMPY_test_image
    "base_dir": "C:\\src\\QDMPY_test_image\\",
    "filepath": "Spin relaxation (T1)_19",
    
    "additional_bins": 0, 

    "system_name": "Liams_Widefield",

    "ROI": "Full", 
    "ROI_start": [10, 10],
    "ROI_end": [200,200],
    
    "AOI_1_start": [10, 10],
    "AOI_1_end": [20, 20], 
    "AOI_2_start": [10, 50],
    "AOI_2_end": [20, 60], 
    "AOI_3_start": [60, 15],
    "AOI_3_end": [70, 35], 
    
    "normalisation": "div",
    
    "fit_pixels": True, 
    "force_fit": True, 
    "auto_match_prev_ROI_options": False,
    
    "fit_backend": "gpufit",
    "fit_backend_comparison": ["scipy", "gpufit"],
    "scramble_pixels": True,
    
    "scipy_sub_threads": 2, 

    "fit_functions":  {"constant": 1, "stretched_exponential": 1},

    "constant_guess": 1,
    "constant_bounds": [0.9, 1.1],

    "power_exp_guess": 0.7,
    "power_exp_bounds": [0.5, 1.0],

    "charac_exp_t_guess": 1e-3,
    "charac_exp_t_bounds": [1e-9, 100e-3],

    "amp_exp_guess": 0.07,
    "amp_exp_bounds": [0.001, 0.2],
    
    "mpl_rcparams": {
        "legend.loc": "upper right" 
    }
}

In [4]:
# QDMPy_path = data_loading.DIR_PATH
# options = data_loading.load_options(path=QDMPy_path / "options/test_options_t1.json", check_for_prev_result=True)
options = data_loading.load_options(options_dict=options_dict, check_for_prev_result=True)
fit_plots.set_mpl_rcparams(options)
raw_data, prelim_sweep_list = data_loading.load_raw_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = data_loading.reshape_dataset(
        options, raw_data, prelim_sweep_list
    )
fig1 = fit_plots.plot_ROI_PL_image(options, PL_image)

C:\src\QDMPy\QDMPy\data_loading.py:567: UserWarning: oiy fix up additional_bins = 0 to be handled more nicely...?
  warnings.warn("oiy fix up additional_bins = 0 to be handled more nicely...?")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
AOIs = data_loading.define_AOIs(options)
fig2 = fit_plots.plot_AOI_PL_images(options, PL_image_ROI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
aoi_spectra_plot = fit_plots.plot_AOI_spectra(options, AOIs, sig, ref, sweep_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit AOI, ROI averages, single pixel

In [7]:
fit_model = fit_interface.define_fit_model(options)
backend_ROI_results_lst = fit_interface.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
fig3 = fit_plots.plot_ROI_avg_fits(options, backend_ROI_results_lst)

C:\src\QDMPy\QDMPy\fit_scipy.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fit_result_collection_lst = fit_interface.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, AOIs, backend_ROI_results_lst)
AOI_spectra_fit_fig = fit_plots.plot_AOI_spectra_fit(options, sig, ref, sweep_list, AOIs, fit_result_collection_lst, fit_model)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit pixels

In [9]:
if (options["force_fit"] or not options["found_prev_result"]) and options["fit_pixels"]:
    pixel_fit_params = fit_interface.fit_pixels(options, sig_norm, sweep_list, fit_model, backend_ROI_results_lst[0])
elif options["found_prev_result"]:
    pixel_fit_params = fit_interface.load_prev_fit_results(options)
else:
    pixel_fit_params = None  # not fitting pixels, this stops plotting (e.g. via plot_param_images) from erroring

# Plot (fit) parameters

In [10]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "constant")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "power_exp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "charac_exp_t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "amp_exp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# fit_plots.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")

In [15]:
data_loading.save_options(options)
options

{'base_dir': 'C:\\src\\QDMPY_test_image\\',
 'filepath': 'C:\\src\\QDMPY_test_image\\Spin relaxation (T1)_19',
 'raw_pixel_size': 2.22e-07,
 'ignore_ref': False,
 'additional_bins': 1,
 'old_binning_convention': False,
 'system_name': 'Liams_Widefield',
 'ROI': 'Full',
 'ROI_start': [10, 10],
 'ROI_end': [200, 200],
 'remove_start_sweep': 1,
 'remove_end_sweep': 0,
 'normalisation': 'div',
 'single_pixel_check': [1, 1],
 'AOI_1_start': (10, 10),
 'AOI_1_end': (20, 20),
 'AOI_2_start': (10, 50),
 'AOI_2_end': (20, 60),
 'AOI_3_start': (60, 15),
 'AOI_3_end': (70, 35),
 'AOI_4_start': None,
 'AOI_4_end': None,
 'AOI_5_start': None,
 'AOI_5_end': None,
 'fit_backend': 'gpufit',
 'fit_backend_comparison': ['scipy', 'gpufit'],
 'force_fit': True,
 'fit_pixels': True,
 'auto_match_prev_ROI_options': False,
 'scramble_pixels': True,
 'use_ROI_avg_fit_res_for_all_pixels': True,
 'fit_functions': {'constant': 1, 'stretched_exponential': 1},
 'pos_guess': None,
 'pos_range': 25,
 'amp_guess': -0